In [3]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'


#this script takes some edgeR table results and analize the expressed genes, try to understad which genes are differentialy expressed among the samples. 
#the analysis based on q_val
import sys
!{sys.executable} -m pip install sklearn

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.decomposition import PCA
import math
import re
import ast
import itertools
import scipy
import os



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## creating summary table for all DE genes and add it to 1-1 ortho metatable : 

one_to_one_dir=r'C:\Users\TzachiHNB1\Documents\docs_from_lilac\tzachi_hagai\my_project_from_22_11\more_oneTOone_analysis'

In [7]:
human_mouse_ortho=pd.read_csv(r'general_input\1-1_human_mouse_metadata_with_old_chip_seq_cols.csv',index_col=0)
human_mouse_ortho

human_mouse_ortho_1_1=human_mouse_ortho[human_mouse_ortho['Mouse homology type']=='ortholog_one2one']
human_mouse_ortho_1_1

mouse_human_orto_dict=dict(zip(human_mouse_ortho_1_1.index,human_mouse_ortho_1_1['Mouse gene stable ID']))
human_mouse_orto_dict=dict(zip(human_mouse_ortho_1_1['Mouse gene stable ID'],human_mouse_ortho_1_1.index))
human_mouse_orto_dict

{'ENSMUSG00000064341': 'ENSG00000198888',
 'ENSMUSG00000064345': 'ENSG00000198763',
 'ENSMUSG00000064351': 'ENSG00000198804',
 'ENSMUSG00000064354': 'ENSG00000198712',
 'ENSMUSG00000064356': 'ENSG00000228253',
 'ENSMUSG00000064357': 'ENSG00000198899',
 'ENSMUSG00000064358': 'ENSG00000198938',
 'ENSMUSG00000064360': 'ENSG00000198840',
 'ENSMUSG00000065947': 'ENSG00000212907',
 'ENSMUSG00000064363': 'ENSG00000198886',
 'ENSMUSG00000064367': 'ENSG00000198786',
 'ENSMUSG00000064368': 'ENSG00000198695',
 'ENSMUSG00000064370': 'ENSG00000198727',
 'ENSMUSG00000065495': 'ENSG00000207782',
 'ENSMUSG00000065424': 'ENSG00000284447',
 'ENSMUSG00000088059': 'ENSG00000252213',
 'ENSMUSG00000065501': 'ENSG00000199043',
 'ENSMUSG00000065208': 'ENSG00000263776',
 'ENSMUSG00000065242': 'ENSG00000239190',
 'ENSMUSG00000065444': 'ENSG00000207808',
 'ENSMUSG00000065159': 'ENSG00000207099',
 'ENSMUSG00000088385': 'ENSG00000212368',
 'ENSMUSG00000065053': 'ENSG00000206605',
 'ENSMUSG00000080486': 'ENSG000002

In [5]:
print(os.getcwd())

C:\Users\TzachiHNB6\Documents\lilach's paper\chip-seq\chip_seq_not_full_dir\proccess chipseq data before the actual analysis\new_chip_seq_preparation_folder


In [8]:
# define the groups: 

study='Hagai'
species=['homo','mus']
if species==['homo','mus']:
    species_full_names=['Homo sapiens','Mus musculus']
treats=['LF4','PIC4']
    
    


df_groups=pd.read_csv(r'general_input\{}_{}_vs_{}_{}_and_{}_groups_results.csv'.format(study,species[0],species[1],treats[0],treats[1]),index_col=0)
df_groups

DE_genes_FC_values_df=pd.read_csv(r'general_input\DE_genes_human_mouse_pic.csv',index_col=1)

DE_genes=DE_genes_FC_values_df.index
non_DE_genes=[i for i in human_mouse_ortho_1_1.index if i not in DE_genes]

# stim groups - high and low
group_stim_high=df_groups[(df_groups[f'group_{treats[1]}']==f'{treats[1]}_high_strict')|(df_groups[f'group_{treats[1]}']==f'{treats[1]}_high_moderate')]
print(len(group_stim_high.index))
group_stim_high_and_DE=[i for i in group_stim_high.index if i in DE_genes_FC_values_df.index]
print(len(group_stim_high_and_DE))


group_stim_low=df_groups[(df_groups[f'group_{treats[1]}']==f'{treats[1]}_low_strict')|(df_groups[f'group_{treats[1]}']==f'{treats[1]}_low_moderate')]
print(len(group_stim_low.index))
group_stim_low_and_DE=[i for i in group_stim_low.index if i in DE_genes_FC_values_df.index]
print(len(group_stim_low_and_DE))


# unst groups - high and low
group_unst_high=df_groups[(df_groups[f'group_{treats[0]}']==f'{treats[0]}_high_strict')|(df_groups[f'group_{treats[0]}']==f'{treats[0]}_high_moderate')]
print(len(group_unst_high.index))
group_unst_high_and_DE=[i for i in group_unst_high.index if i in DE_genes_FC_values_df.index]
print(len(group_unst_high_and_DE))


group_unst_low=df_groups[(df_groups[f'group_{treats[0]}']==f'{treats[0]}_low_strict')|(df_groups[f'group_{treats[0]}']==f'{treats[0]}_low_moderate')]
print(len(group_unst_low.index))
group_unst_low_and_DE=[i for i in group_unst_low.index if i in DE_genes_FC_values_df.index]
print(len(group_unst_low_and_DE))


579
224
519
166
498
104
476
84


In [19]:
mouse_human_orto_dict

{'ENSG00000198888': 'ENSMUSG00000064341',
 'ENSG00000198763': 'ENSMUSG00000064345',
 'ENSG00000198804': 'ENSMUSG00000064351',
 'ENSG00000198712': 'ENSMUSG00000064354',
 'ENSG00000228253': 'ENSMUSG00000064356',
 'ENSG00000198899': 'ENSMUSG00000064357',
 'ENSG00000198938': 'ENSMUSG00000064358',
 'ENSG00000198840': 'ENSMUSG00000064360',
 'ENSG00000212907': 'ENSMUSG00000065947',
 'ENSG00000198886': 'ENSMUSG00000064363',
 'ENSG00000198786': 'ENSMUSG00000064367',
 'ENSG00000198695': 'ENSMUSG00000064368',
 'ENSG00000198727': 'ENSMUSG00000064370',
 'ENSG00000207782': 'ENSMUSG00000065495',
 'ENSG00000284447': 'ENSMUSG00000065424',
 'ENSG00000252213': 'ENSMUSG00000088059',
 'ENSG00000199043': 'ENSMUSG00000065501',
 'ENSG00000263776': 'ENSMUSG00000065208',
 'ENSG00000239190': 'ENSMUSG00000065242',
 'ENSG00000207808': 'ENSMUSG00000065444',
 'ENSG00000207099': 'ENSMUSG00000065159',
 'ENSG00000212368': 'ENSMUSG00000088385',
 'ENSG00000206605': 'ENSMUSG00000065053',
 'ENSG00000221500': 'ENSMUSG000000

In [26]:
# for DE genes

In [17]:
#peaks_of=['chip_seq','atac']
peaks='chip_seq'
species=['human','mouse']
#promoters=['prom1000100','prom2000500']
promoters=['prom1M']
intersects=['00','03','05']

if peaks=='chip_seq':
    histon=['H3K27ac','H3K4me3']
    treats=['PIC4','UNST']
elif peaks=='atac':
    histon=['atac']
    treats=['0h','4h','8h']

res_path=r'summary_{}_{}_{}_{}_DE_genes.csv'.format(peaks,study,species[0],species[1])
print(res_path)
res_df=pd.DataFrame(index=DE_genes)
res_df_all=pd.DataFrame(index=human_mouse_ortho_1_1.index)

res_df_all
res_df    


summary_chip_seq_Hagai_human_mouse_DE_genes.csv


""
0
ENSG00000198888
ENSG00000198763
ENSG00000198804
ENSG00000198712
ENSG00000228253
...
ENSG00000169946
ENSG00000094661
ENSG00000114770


In [18]:
res_df

""
0
ENSG00000198888
ENSG00000198763
ENSG00000198804
ENSG00000198712
ENSG00000228253
...
ENSG00000169946
ENSG00000094661
ENSG00000114770


In [22]:
# for all genes 
for s in species:
    print(s)
    for p in promoters:
        for h in histon:
            for intr in intersects:
                for t in treats:
                    #print(t)
                    file_path=r'results_from_cluster\bedtools_intersect\{}\{}\{}\{}peaks_intersect_{}.txt'.format(s,p,h,t,intr)
                    print(file_path)
                    df_curr=pd.read_csv(file_path, sep='\t', names=['crom','start','end','gene'])
                    #print(df_curr)
                    
                    col_name=r'{}_{}_{}_{}peaks_intersect_{}'.format(s,p,h,t,intr)
                    
                    
                    if s=='human':
                        for row in res_df.index:
                            if (row in mouse_human_orto_dict.keys()) and (row in list(df_curr['gene'])) :
                                #print(row)
                                res_df.at[row,col_name]=1
                            else:
                                res_df.at[row,col_name]=0
                    
                    elif s=='mouse':
                        for row in res_df.index:
                           # print(row)
                            if (row in mouse_human_orto_dict.keys()) and (mouse_human_orto_dict[row] in list(df_curr['gene'])):
                                res_df.at[row,col_name]=1
                            else:
                                res_df.at[row,col_name]=0


res_df=res_df.astype('int32')

res_df.to_csv(res_path)
res_df
                    

                    
                   

human
results_from_cluster\bedtools_intersect\human\prom1M\H3K27ac\PIC4peaks_intersect_00.txt
results_from_cluster\bedtools_intersect\human\prom1M\H3K27ac\UNSTpeaks_intersect_00.txt
results_from_cluster\bedtools_intersect\human\prom1M\H3K27ac\PIC4peaks_intersect_03.txt
results_from_cluster\bedtools_intersect\human\prom1M\H3K27ac\UNSTpeaks_intersect_03.txt
results_from_cluster\bedtools_intersect\human\prom1M\H3K27ac\PIC4peaks_intersect_05.txt
results_from_cluster\bedtools_intersect\human\prom1M\H3K27ac\UNSTpeaks_intersect_05.txt
results_from_cluster\bedtools_intersect\human\prom1M\H3K4me3\PIC4peaks_intersect_00.txt
results_from_cluster\bedtools_intersect\human\prom1M\H3K4me3\UNSTpeaks_intersect_00.txt
results_from_cluster\bedtools_intersect\human\prom1M\H3K4me3\PIC4peaks_intersect_03.txt
results_from_cluster\bedtools_intersect\human\prom1M\H3K4me3\UNSTpeaks_intersect_03.txt
results_from_cluster\bedtools_intersect\human\prom1M\H3K4me3\PIC4peaks_intersect_05.txt
results_from_cluster\bedto

,human_prom1M_H3K27ac_PIC4peaks_intersect_00,human_prom1M_H3K27ac_UNSTpeaks_intersect_00,human_prom1M_H3K27ac_PIC4peaks_intersect_03,human_prom1M_H3K27ac_UNSTpeaks_intersect_03,human_prom1M_H3K27ac_PIC4peaks_intersect_05,human_prom1M_H3K27ac_UNSTpeaks_intersect_05,human_prom1M_H3K4me3_PIC4peaks_intersect_00,human_prom1M_H3K4me3_UNSTpeaks_intersect_00,human_prom1M_H3K4me3_PIC4peaks_intersect_03,human_prom1M_H3K4me3_UNSTpeaks_intersect_03,...,mouse_prom1M_H3K27ac_PIC4peaks_intersect_03,mouse_prom1M_H3K27ac_UNSTpeaks_intersect_03,mouse_prom1M_H3K27ac_PIC4peaks_intersect_05,mouse_prom1M_H3K27ac_UNSTpeaks_intersect_05,mouse_prom1M_H3K4me3_PIC4peaks_intersect_00,mouse_prom1M_H3K4me3_UNSTpeaks_intersect_00,mouse_prom1M_H3K4me3_PIC4peaks_intersect_03,mouse_prom1M_H3K4me3_UNSTpeaks_intersect_03,mouse_prom1M_H3K4me3_PIC4peaks_intersect_05,mouse_prom1M_H3K4me3_UNSTpeaks_intersect_05
0,,,,,,,,,,,,,,,,,,,,,
ENSG00000198888,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000198763,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000198804,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000198712,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000228253,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000169946,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
ENSG00000094661,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
ENSG00000114770,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [15]:
# for all 1-1

In [23]:
#peaks_of=['chip_seq','atac']
peaks='chip_seq'
species=['human','mouse']
#promoters=['prom1000100','prom2000500']
promoters=['prom1M']

intersects=['00','03','05']

if peaks=='chip_seq':
    histon=['H3K27ac','H3K4me3']
    treats=['PIC4','UNST']
elif peaks=='atac':
    histon=['atac']
    treats=['0h','4h','8h']

res_path=r'summary_{}_{}_{}_{}_all_1_1.csv'.format(peaks,study,species[0],species[1])
print(res_path)
res_df=pd.DataFrame(index=DE_genes)
res_df_all=pd.DataFrame(index=human_mouse_ortho_1_1.index)

res_df    
res_df_all
res_df=res_df_all.copy()
res_df

summary_chip_seq_Hagai_human_mouse_all_1_1.csv


""
ENSG00000198888
ENSG00000198763
ENSG00000198804
ENSG00000198712
ENSG00000228253
...
ENSG00000125740
ENSG00000139648
ENSG00000185614
ENSG00000198400


In [24]:
# for all genes 
for s in species:
    print(s)
    for p in promoters:
        for h in histon:
            for intr in intersects:
                for t in treats:
                    #print(t)
                    file_path=r'results_from_cluster\bedtools_intersect\{}\{}\{}\{}peaks_intersect_{}.txt'.format(s,p,h,t,intr)
                    print(file_path)
                    df_curr=pd.read_csv(file_path, sep='\t', names=['crom','start','end','gene'])
                    
                    
                    col_name=r'{}_{}_{}_{}peaks_intersect_{}'.format(s,p,h,t,intr)
                    
                    
                    if s=='human':
                        for row in res_df.index:
                            if (row in mouse_human_orto_dict.keys()) and (row in list(df_curr['gene'])) :
                                #print(row)
                                res_df.at[row,col_name]=1
                            else:
                                res_df.at[row,col_name]=0
                    
                    elif s=='mouse':
                        for row in res_df.index:
                           # print(row)
                            if (row in mouse_human_orto_dict.keys()) and (mouse_human_orto_dict[row] in list(df_curr['gene'])):
                                res_df.at[row,col_name]=1
                            else:
                                res_df.at[row,col_name]=0


res_df=res_df.astype('int32')

res_df.to_csv(res_path)
res_df
                    

                    
                   

human
results_from_cluster\bedtools_intersect\human\prom1M\H3K27ac\PIC4peaks_intersect_00.txt


KeyboardInterrupt: 